# Pathway to Pathway (Reactome)

In [3]:
import pandas as pd 
import numpy as np
import csv
import json
import biomedkg_utils
from biomedkg_utils import *
import matplotlib.pyplot as plt
from matplotlib_venn import venn3, venn2

### Human Reactome Pathways


In [8]:
pw_prefix = 'Reactome_Pathway:'
headers = ['Pathway (Reactome)','Pathway (Reactome)', 'Relationship', 'Score']
#os.system('wget -N -P ../data/ https://reactome.org/download/current/ReactomePathwaysRelation.txt')

pw_tree_df = pd.read_table('input/ReactomePathwaysRelation.txt', header=None)
pw_tree_df.columns = ['Parent', 'Child']
human_pw_tree_df = pw_tree_df[pw_tree_df['Parent'].str.contains('-HSA-')].copy()
human_pw_tree_df['Parent'] = [pw_prefix+pw for pw in human_pw_tree_df['Parent']]
human_pw_tree_df['Child'] = [pw_prefix+pw for pw in human_pw_tree_df['Child']]
relations = ['-pathway_is_parent_of->']*len(human_pw_tree_df)
scores = [1.0]*len(human_pw_tree_df)
human_pw_tree_df.insert(2, 'relation', relations)
human_pw_tree_df.insert(3, 'scores', scores)
human_pw_tree_df.columns = headers
human_pw_tree_df

human_pw_tree_df.to_csv('output/pathway2pathway/edges_reactomePathwayHierarchy.csv', index = False)
human_pw_tree_df.to_csv('output/edges to use/Pathway_(Reactome)_2_Pathway_(Reactome).csv', index = False)
df = pd.read_csv('output/pathway2pathway/edges_reactomePathwayHierarchy.csv')
df.tail()

,Pathway (Reactome),Pathway (Reactome).1,Relationship,Score
2623,Reactome_Pathway:R-HSA-983705,Reactome_Pathway:R-HSA-983695,-pathway_is_parent_of->,1.0
2624,Reactome_Pathway:R-HSA-983712,Reactome_Pathway:R-HSA-2672351,-pathway_is_parent_of->,1.0
2625,Reactome_Pathway:R-HSA-983712,Reactome_Pathway:R-HSA-936837,-pathway_is_parent_of->,1.0
2626,Reactome_Pathway:R-HSA-991365,Reactome_Pathway:R-HSA-170670,-pathway_is_parent_of->,1.0
2627,Reactome_Pathway:R-HSA-991365,Reactome_Pathway:R-HSA-997272,-pathway_is_parent_of->,1.0


# Pathway to Pathway (KEGG)

In [5]:
! curl https://rest.kegg.jp/list/pathway/hsa > ./input/KEGG/human_pathways.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20860    0 20860    0     0   1418      0 --:--:--  0:00:14 --:--:--  5247


In [6]:
def check_if_pathway_category(pathway_name):
    '''
    FUNCTION:
    - Some 'pathway names' are categories that begin with a decimal number.
      Some pathway names begin with the pathway ID. We want to determine
      which 'pathway names' are actually categories. We will use these as categories.
      The other pathway names will be included if they are human pathways.
    
    PARAMS:
    - pathway_name: The alleged 'pathway name' which includes prefixes.
    '''
    first_word = pathway_name.split(' ')[0]
    
    # Check if the first word is a decimal number (e.g., 1.1 Global and overview maps)
    
    # Decimal in first word?
    if '.' not in first_word:
        return False
    else:
        
        # First word == section or ID?
        try:
            first_word_float = float(first_word)
            return True
        except:
            return False

def process_pathway_name(orig_name):
    '''
    FUNCTION:
    - Pathway names sometimes have prefixes (e.g., 00010 or M or N)
      We want to remove them and just get the pathway name.
      
    PARAMS:
    - orig_name (str): The pathway name with the prefix
    '''
    orig_name = orig_name.split(' ')
    
    for word_ind, word in enumerate(orig_name):
        # Remove number prefix
        try: word = float(word); continue
        except: pass
    
        # Remove single letter prefixes
        if len(word) == 1: continue
        
        cleaned_name = ' '.join(orig_name[word_ind:]).strip()
        return cleaned_name
    return _

In [7]:
'''Align Pathway ID -is- Pathway Name'''
pathway_id2name = dict()
pathway_name2id = dict()

for line in open('input/KEGG/kegg_human_pathway_to_pathway_id.tsv'):
    line = line.strip().split('\t')
    
    # Pathway ID, Pathway Name
    pathway_id = line[0].replace('path:','path_')
    pathway_name = process_pathway_name(line[1])
    
    # Pathway Name -is- Pathway ID
    pathway_id2name[pathway_id] = pathway_name
    pathway_name2id[pathway_name] = pathway_id

In [8]:
''' Pathway -parent of-> Pathway '''
parent2child_kegg_pathway = dict()

for line in open('input/KEGG/kegg_pathway_hierchy.tsv'):
    line = line.strip().split('\t')
    
    # Pathways' Names
    parent_pathway_name = line[0].replace('path:','path_')
    child_pathway_name  = line[1].strip().replace('path:','path_')
    
    # Is the parent pathway good (human or a category)?
    cleaned_parent_pathway_name = process_pathway_name(parent_pathway_name)
    if cleaned_parent_pathway_name in pathway_name2id: 
        parent_human_pathway = True
        parent_pathway_category = False
    else:
        parent_human_pathway = False
        parent_pathway_category = check_if_pathway_category(parent_pathway_name)
    
    if parent_pathway_category:
        parent_pathway_id = parent_pathway_name
    elif parent_human_pathway:
        parent_pathway_id = pathway_name2id[cleaned_parent_pathway_name]
    else:
        continue
    
    
    # Is the child pathway good (human or a category)?
    cleaned_child_pathway_name = process_pathway_name(child_pathway_name)
    if cleaned_child_pathway_name in pathway_name2id:
        child_human_pathway = True
        child_pathway_category = False
    else:
        child_human_pathway = False
        child_pathway_category = check_if_pathway_category(child_pathway_name)

    if child_pathway_category:
        child_pathway_id = child_pathway_name
    elif child_human_pathway:
        child_pathway_id = pathway_name2id[cleaned_child_pathway_name]
    else:
        continue
        
    # Parent Pathway - Child Pathway
    parent2child_kegg_pathway.setdefault(parent_pathway_id, set()).add(child_pathway_id)
    
    
# Output edges    
file = 'Pathway_(KEGG)_2_Pathway_(KEGG).csv'
outpath = os.path.join('output/pathway2pathway/', file)
output_edgefile_onerel_noweight(
    outpath = outpath,
    columns = ['Pathway (KEGG)','Pathway (KEGG)','Relationship'],
    dictionary = parent2child_kegg_pathway,
    rel = '-parent_of->',
    prefix_col1='KEGG_Pathway:',
    prefix_col2='KEGG_Pathway:'
)
df = pd.read_csv(outpath)
df.to_csv(os.path.join('output/edges', file), index=False)
df.to_csv(os.path.join('output/edges to use/', file), index=False)

## Pathway Alignment Attempts
Not much happened. Most pathways couldn't be aligned between databases, even when really non-strict rules were used for considering pathways as similar (e.g., if they shared half their proteins)

In [9]:
gene2kegg_pathway_df = pd.read_csv(open('output/pathway2gene/edges_gene2KEGGPathway.csv'))[['Gene (Entrez)','Pathway (KEGG)']]
gene2reactome_pathway_df = pd.read_csv(open('output/pathway2gene/edges_gene2reactomePathway.csv'))[['Gene (Entrez)','Pathway (Reactome)']]

def get_pathway2gene_dict_from_df(df):
    '''
    FUNCTION:
    - Map pathways to genes into a dict using a dataframe as input. 
    
    PARAMS:
    - df: dataframe with gene to pathway edges
    '''
    gene2pathway = dict()
    pathway2gene = dict()

    col = df.columns
    for index in range(0,len(df)):
        gene = df[col[0]].iloc[index]
        pathway = df[col[1]].iloc[index]

        gene2pathway.setdefault(gene, set()).add(pathway)
        pathway2gene.setdefault(pathway, set()).add(gene)
        
    return gene2pathway, pathway2gene


gene2kegg_pathway, kegg_pathway2gene = get_pathway2gene_dict_from_df(gene2kegg_pathway_df)
gene2reactome_pathway, reactome_pathway2gene = get_pathway2gene_dict_from_df(gene2reactome_pathway_df)
smpdb_pathway2gene = switch_dictlist_to_dictset(json.load(open('output/pathway2gene/smpdb_pathway2gene.json')))
gene2smpdb_pathway = switch_dictlist_to_dictset(json.load(open('output/pathway2gene/gene2smpdb_pathway.json')))

In [ ]:
def jaccard_similar_two_sets(shared_genes, all_genes, SIMILARITY_THRESHOLD):
    '''
    FUNCTION:
    - Determine if two sets have a jaccard similarity above a certain threshold
    '''
    jaccard_similarity = len(shared_genes)/len(all_genes)
    if jaccard_similarity > SIMILARITY_THRESHOLD:
        return True
    else:
        return False


# KEGG
for kegg_pathway, keggs_genes in kegg_pathway2gene.items():
    
    # Reactome
    for reactome_pathway, reactomes_genes in reactome_pathway2gene.items():
        
        all_genes_kr = keggs_genes.union(reactomes_genes)
        shared_genes_kr = keggs_genes.intersection(reactomes_genes)
        kegg_reactome_similar = jaccard_similar_two_sets(shared_genes_kr, all_genes_kr, 0.8)
        
        if kegg_reactome_similar:
            print(kegg_pathway+'|'+reactome_pathway)
            #kegg_ids_for_venn.add(kegg_pathway+'|'+reactome_pathway)
            #reactome_ids_for_venn.add(kegg_pathway+'|'+reactome_pathway)

        
        # SMPDB 
        for smpdb_pathway, smpdb_genes in smpdb_pathway2gene.items():
            
            # Intersection genes
            shared_genes_ks = keggs_genes.intersection(smpdb_genes)
            shared_genes_sr = smpdb_genes.intersection(reactomes_genes)
            
            # Union genes
            all_genes_ks = keggs_genes.union(smpdb_genes)
            all_genes_sr = smpdb_genes.union(reactomes_genes)

            # Jaccard similarity
            kegg_smpdb_similar = jaccard_similar_two_sets(shared_genes_ks, all_genes_ks, 0.8)
            smpdb_reactome_similar = jaccard_similar_two_sets(shared_genes_sr, all_genes_sr, 0.8)
            

            if kegg_smpdb_similar:
                print(kegg_pathway+'|'+smpdb_pathway)
                #kegg_ids_for_venn.add(kegg_pathway+'|'+smpdb_pathway)
                #smpdb_ids_for_venn.add(kegg_pathway+'|'+smpdb_pathway)
                
            if smpdb_reactome_similar:
                print(reactome_pathway+'|'+smpdb_pathway)
                #reactome_ids_for_venn.add(reactome_pathway+'|'+smpdb_pathway)
                #smpdb_ids_for_venn.add(reactome_pathway+'|'+smpdb_pathway)


In [10]:
for kegg_pathway, keggs_genes in kegg_pathway2gene.items():
    for reactome_pathway, reactomes_genes in reactome_pathway2gene.items():
        shared_genes = keggs_genes.intersection(reactomes_genes)
        all_genes = keggs_genes.union(reactomes_genes)
        jaccard_similarity = len(shared_genes)/len(all_genes)
        if jaccard_similarity > 0.8:
            print(kegg_pathway, reactome_pathway)


KEGG_Pathway:path:hsa04740 Reactome_Pathway:R-HSA-9752946
KEGG_Pathway:path:hsa03050 Reactome_Pathway:R-HSA-211733
KEGG_Pathway:path:hsa03050 Reactome_Pathway:R-HSA-1236978


In [11]:
for kegg_pathway, keggs_genes in kegg_pathway2gene.items():
    for smpdb_pathway, smpdbs_genes in smpdb_pathway2gene.items():
        shared_genes = keggs_genes.intersection(smpdbs_genes)
        all_genes = keggs_genes.union(smpdbs_genes)
        jaccard_similarity = len(shared_genes)/len(all_genes)
        if jaccard_similarity > 0.8:
            print(kegg_pathway, smpdb_pathway)


In [12]:
for smpdb_pathway, smpdbs_genes in smpdb_pathway2gene.items():
    for reactome_pathway, reactomes_genes in reactome_pathway2gene.items():
        shared_genes = smpdbs_genes.intersection(reactomes_genes)
        all_genes = smpdbs_genes.union(reactomes_genes)
        jaccard_similarity = len(shared_genes)/len(all_genes)
        if jaccard_similarity > 0.8:
            print(smpdb_pathway, reactome_pathway)

# SMPDB Pathways: Text Descriptions
add to name file 

In [2]:
smpdb_dfs = pd.read_csv('input/smpdb_files/smpdb_pathways.csv')

In [22]:
smpdb_pathway2description = dict()

for i in range(len(smpdb_dfs)):
    
    # SMPDB Pathway ID, Description
    smpdb_id = smpdb_dfs['SMPDB ID'].iloc[i]
    descript = smpdb_dfs['Description'].iloc[i]
    
    # SMPDB Pathway ID to Description
    smpdb_pathway2description[smpdb_id] = descript
    

# Export
json.dump(smpdb_pathway2description, open('output/pathway2pathway/smpdb_pathway2description.json','w'))
json.dump(smpdb_pathway2description, open('output/nodes/node features/node2text/smpdb_pathway2description.json','w'))